In [44]:
import h5py
import os
import sys
sys.path.append(r'C:\Users\kid\SynologyDrive\Lab2023\KIDs\QTLab2324\IR_SING_PHOT')
from HDF5 import HDF5
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter, find_peaks
from scipy.optimize import curve_fit
from iminuit import Minuit
from iminuit.cost import LeastSquares

In [45]:
def read(parent):
    data = []
    counter = 0
    for root, dirs, files in os.walk(parent):
        for file in files:
            if file.endswith('.hdf5') and counter<10:
                file_path = os.path.join(root, file)
                f = HDF5.HDF5()
                f.name = file_path
                data.append(f.load_hdf5()['Signals']['I'])
                counter = counter + 1
                print(f'reading file : {counter}')
    return data


In [47]:
data = read(r'C:\Users\kid\SynologyDrive\Lab2023\KIDs\QTLab2324\IR_SING_PHOT\RUNS\10')

Execution time: 0.172
reading file : 1
Execution time: 0.164
reading file : 2
Execution time: 0.168
reading file : 3
Execution time: 0.158
reading file : 4
Execution time: 0.167
reading file : 5
Execution time: 0.163
reading file : 6
Execution time: 0.172
reading file : 7
Execution time: 0.159
reading file : 8
Execution time: 0.161
reading file : 9
Execution time: 0.16
reading file : 10


In [48]:
def model(x, A,B,C,D):
    return A*np.exp(-B*(x-C))+D

def sigmoid(x, A, B, C, D):
    return A / (1.0 + np.exp(-B * (x - C))) + D

In [43]:
for i in range(len(data)):
    xmin, ymin = [np.argmin(data[i]) , min(data[i])]
    I = -data[i][xmin-3000:xmin+10000]
    T = np.linspace(xmin-3000,xmin+10000,len(I),dtype=int)
    # Apply moving average filter
    window_size = 50  # Size of the moving average window
    cumsum = np.cumsum(I)
    cumsum[window_size:] = cumsum[window_size:] - cumsum[:-window_size]
    moving_avg = cumsum[window_size - 1:] / window_size
    # Pad the moving average to match the length of the original signal
    moving_avg = np.pad(moving_avg, (window_size - 1, 0), mode='constant', constant_values=(moving_avg[0], moving_avg[-1]))
    window_length = 171  # Must be odd
    polyorder = 3
    sg_filtered_signal = savgol_filter(moving_avg, window_length, polyorder)

    # Post-processing: Optional further smoothing
    window_size = 5
    smoothed_signal = np.convolve(sg_filtered_signal, np.ones(window_size)/window_size, mode='valid')
    window_length = 171  # Must be odd
    polyorder = 3
    sg_filtered_signal = savgol_filter(moving_avg, window_length, polyorder)

    # Post-processing: Optional further smoothing
    window_size = 5
    smoothed_signal = np.convolve(sg_filtered_signal, np.ones(window_size)/window_size, mode='valid')

    len_fit = int(np.floor(len(I)/10))
    pol_amp = np.polyfit(T[:len_fit], I[:len_fit], 2)
    Mean = np.mean(np.polyval(pol_amp, T[:len_fit])-I[:len_fit])
    rms = np.sqrt(np.mean((np.polyval(pol_amp, T[:len_fit])-I[:len_fit])**2))   

    p0 = [max(smoothed_signal),0.00046,267200,Mean]
    p0_sig = [max(smoothed_signal),0.1,267200,Mean]

    sig = LeastSquares(T[2000:3200],smoothed_signal[2000:3200],rms,sigmoid)
    exp = LeastSquares(T[3000:len(smoothed_signal)],smoothed_signal[3000:len(smoothed_signal)],rms,model)
    msig = Minuit(sig,*p0_sig)
    msig.migrad(iterate=1000)
    
    if msig.accurate:
        mexp = Minuit(exp,*p0)
        
        mexp.migrad(iterate=1000)

        # Calculate the 10% and 90% points of the fitted signal
        y_10 = msig.values['D'] + 0.1 * msig.values['A']
        y_90 = msig.values['D'] + 0.9 * msig.values['A']

        # Find the time at which the signal reaches these values
        t_10 = T[np.abs(sigmoid(T[2000:3200],*msig.values) - y_10).argmin()]
        t_90 = T[np.abs(sigmoid(T[2000:3200],*msig.values) - y_90).argmin()]

        # Calculate the rise time
        rise_time = (t_90 - t_10)*1e-8
        print(f"Rise Time: {rise_time*1e6:.4f} micro seconds")

        fitted_signal = np.array(list(sigmoid(T[0:3000],*msig.values))+list(model(T[3000:],*mexp.values)))

        window = 111 
        polyorder = 3 
        sd = savgol_filter(fitted_signal,window, polyorder, deriv=2) 

        th = 4*1e-8
        peaks, props = find_peaks(sd, th)
        peak = peaks[np.argmax(props['peak_heights'])]

        with open('Trigger_RUN_10.txt','a') as file:
            file.writelines('\n'+str(peaks[0])+'\t'+str(props['peak_heights'][0]))
    
    else: 
        print('Could not fit...')
        pass

Could not fit...


C:\Users\kid\AppData\Local\Temp\ipykernel_1492\1250387595.py:5: RuntimeWarning: overflow encountered in exp
  return A / (1.0 + np.exp(-B * (x - C))) + D


Could not fit...
Could not fit...
Could not fit...
Could not fit...
Could not fit...
Could not fit...
Could not fit...
Could not fit...
Could not fit...
